In [7]:
import pandas as pd
import numpy as np
from collections import defaultdict
import requests
from bs4 import BeautifulSoup
import time

In [8]:
def key_metric_extraction(soup_boxes, real_estate_info, incomplete_idx):
    
    for i, box in enumerate(soup_boxes):
        try:
            address = box.find('address').text[:(-7-5-len(state+city))]
            real_estate_info['address'].append(address)
        except: 
            real_estate_info['address'].append(np.nan)
            incomplete_idx.append(i)
        try:
            zip_code = box.find('address').text[-7:]
            real_estate_info['zip_code'].append(zip_code)
        except: 
            real_estate_info['zip_code'].append(np.nan)
            incomplete_idx.append(i)        
        try:
            price = box.find('span', {'class':'bp-Homecard__Price--value'}).text
            real_estate_info['price'].append(price)
        except: 
            real_estate_info['price'].append(np.nan)
            incomplete_idx.append(i)
        try:
            bed = box.find('span', {'class':'bp-Homecard__Stats--beds text-nowrap'}).text
            real_estate_info['bed'].append(bed)
        except: 
            real_estate_info['bed'].append(np.nan)
            incomplete_idx.append(i)   
        try:
            bath = box.find('span', {'class':'bp-Homecard__Stats--baths text-nowrap'}).text
            real_estate_info['bath'].append(bath)
        except: 
            real_estate_info['bath'].append(np.nan)
            incomplete_idx.append(i)   
        try:
            sqr_footage = box.find('span', {'class':'bp-Homecard__LockedStat--value'}).text
            real_estate_info['sqr_footage'].append(sqr_footage)
        except:
            real_estate_info['sqr_footage'].append(np.nan)
            incomplete_idx.append(i)   
        try:
            property_link = "https://www.redfin.com"+box.find("a").get('href')
            real_estate_info['property_link'].append(property_link)
        except:
            real_estate_info['property_link'].append(np.nan)
            incomplete_idx.append(i)

    return real_estate_info, incomplete_idx


In [ ]:
def crawling_redfin(head, state, city, page):
    
    target_url = f"https://www.redfin.ca/{state}/{city}/page-{page}"
    resp = requests.get(target_url,headers=head,verify=False)
    
    if resp.status_code != 200:
        raise Exception("Failing in webpage requests")

    soup=BeautifulSoup(resp.text,'html.parser')
    soup_boxes = soup.find_all("div",{"class":"HomeCardContainer"})

    return  soup_boxes

In [31]:
head = head={"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"}
state, city = 'bc', 'vancouver'
real_estate_info = defaultdict(list)
incomplete_idx = []

In [ ]:
for i in range(1, 9):
    soup_boxes = crawling_redfin(head, state, city, page = i)
    real_estate_info, incomplete_idx = key_metric_extraction(soup_boxes, real_estate_info, incomplete_idx)
    time.sleep(5)

/Users/chrisgao/miniforge3/envs/web_scraping/lib/python3.12/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.redfin.ca'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/Users/chrisgao/miniforge3/envs/web_scraping/lib/python3.12/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.redfin.ca'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/Users/chrisgao/miniforge3/envs/web_scraping/lib/python3.12/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.redfin.ca'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#t

In [34]:
pd.DataFrame(real_estate_info)

,address,zip_code,price,bed,bath,sqr_footage,property_link
0,1128 W 55th Ave,V6P 1R1,"$5,980,000",6 beds,6.5 baths,"6,130",https://www.redfin.com/bc/vancouver/1128-W-55t...
1,68 Smithe St #1817,V6B 0P4,"$655,000",1 bed,1 bath,501,https://www.redfin.com/bc/vancouver/68-Smithe-...
2,3119 Charles St #1,V5K 3B7,"$1,439,000",3 beds,2.5 baths,"1,217",https://www.redfin.com/bc/vancouver/3119-Charl...
3,1133 Homer St #310,V6B 0B1,"$650,000",1 bed,1 bath,549,https://www.redfin.com/bc/vancouver/1133-Homer...
4,1010 W 42nd Ave #205,V6M 2A8,"$699,000",1 bed,1.5 baths,846,https://www.redfin.com/bc/vancouver/1010-W-42n...
...,...,...,...,...,...,...,...
75,5650 Hampton Pl #14,V6T 2G5,"$2,199,000",3 beds,2.5 baths,"2,824",https://www.redfin.com/bc/greater-vancouver-re...
76,1939 W 12th Ave,V6J 2G1,"$1,099,000",2 beds,1.5 baths,"1,060",https://www.redfin.com/bc/vancouver/1939-W-12t...
77,6616 Wiltshire St,V6P 5G7,"$5,280,000",6 beds,6.5 baths,"4,391",https://www.redfin.com/bc/vancouver/6616-Wilts...
78,5389 Vine St #7,V6M 3Z7,"$1,798,000",3 beds,2 baths,"1,498",https://www.redfin.com/bc/vancouver/5389-Vine-...
